In [1]:
import requests
import json
from typing import List, Any

In [40]:
def callGpt(
        userText: str,
        messages: List[Any],
        temperature: float = 0.7,
        max_tokens: int = 500
    ) -> str:
    url: str = f"https://model.hsueh.tw/callapi/chatGPT?temperature={temperature}&max_tokens={max_tokens}&top_p=0.95&frequency_penalty=0&presence_penalty=0&stop=""&past_messages=3&purpose=none"
    payload = json.dumps(messages)
    headers = {
        'Content-Type': 'application/json',
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    try:
        return str(response.json()['choices'][0]['message']['content'])
    except:
        return "ERROR:"+str(response.text)

In [45]:
def callGPT_keywords_extract(userText: str) -> str:
        return callGpt(userText, [
                {
                    "role": "system",
                    "content": """
                    你是一名台灣國小自然科學領域的老師，你發起了國小自然科學探究活動的題目，你現在必須提取出和探究題目有關以及學生在探究過程中需要聚焦的10個重要的關鍵字，請以台灣國小自然科學課綱會出現的詞彙為主，除關鍵字之外請不要回覆其他訊息。注意，請以'、'區隔，不要出現其他格式的訊息。
                    """
                },
                {"role": "user", "content": userText}
        ],0.3)

In [46]:
activity_topic = """太陽光為什麼可以發電？"""
keywords_topic = callGPT_keywords_extract(activity_topic)
print(keywords_topic)

太陽能、光電效應、太陽能板、能量轉換、環保、可再生能源、發電原理、電能儲存、太陽能利用、綠色能源。


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 两组关键字数据
keywords1 = "太陽能、光電效應、太陽能板、能量轉換、環保、可再生能源、發電原理、電能儲存、太陽能利用、綠色能源"
keywords2 = "宇宙、太空梭、衛星、黑洞、地心引力、蟲洞、暗物質、太陽"

# 合并两组关键字数据为文档列表
documents = [keywords1, keywords2]

# 创建TF-IDF向量化器
tfidf_vectorizer = TfidfVectorizer()

# 转换文档为TF-IDF向量
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# 计算余弦相似度
similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# 打印相似度分数
print("相似度分数：", similarities[0][0])


相似度分数： 0.0


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 两组关键字数据
keywords1 = "太陽能、光電效應、太陽能板、能量轉換、環保"
keywords2 = "太陽能、太陽能板、發電、光能、轉換效率"

# 将关键字数据分割成单词列表
keywords1_list = keywords1.split('、')
keywords2_list = keywords2.split('、')

# 创建TF-IDF向量化器
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([keywords1, keywords2])

# 计算余弦相似度矩阵
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# 提取keywords1和keywords2之间的相似度
similarity_matrix = cosine_sim_matrix[0, 1:]

# 创建一个相似度矩阵
similarity_matrix = np.reshape(similarity_matrix, (1, -1))

# 创建一个矩阵，其中包含keyword1中的每个词与keyword2中的每个词之间的相似度
similarity_matrix = np.outer(tfidf_matrix.toarray()[0], tfidf_matrix.toarray()[1])

# 打印相似度矩阵
print("相似度矩阵：")
print(similarity_matrix)


相似度矩阵：
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [0.08464773 0.08464773 0.08464773 0.0846

In [75]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # 两组关键字数据
# keywords1 = "太陽能、光電效應、太陽能板、能量轉換、環保"
# keywords2 = "太陽能、太陽能板、發電、光能、轉換效率"

# # 将关键字数据分割成单词列表
# keywords1_list = keywords1.split('、')
# keywords2_list = keywords2.split('、')

# # 创建一个5x5的矩阵来存储相似度值
# similarity_matrix = np.zeros((5, 5))

# # 创建TF-IDF向量化器
# tfidf_vectorizer = TfidfVectorizer()

# # 计算每个词在keywords2与keywords1之间的相似度
# for i in range(5):
#     for j in range(5):
#         # 转换为TF-IDF向量
#         tfidf_matrix = tfidf_vectorizer.fit_transform([keywords1_list[j], keywords2_list[i]])
#         # 提取keywords1与keywords2之间的相似度
#         similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
#         # 存储相似度值到矩阵
#         similarity_matrix[i, j] = similarity

# # 打印相似度矩阵
# print("相似度矩阵：")
# print(similarity_matrix)


相似度矩阵：
[[1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [8]:
!pip install -U sentence_transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached torchvision-0.16.0-cp310-cp310-win_amd64.whl (1.3 MB)


In [45]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# 多个句子（关键词）
keywords1 = "太陽能、光電效應、太陽能板、能量轉換、環保"
keywords2 = "太陽能發電、光能、地球暖化、熱、狗"

# 将关键词分割成单词列表
keywords1_list = keywords1.split('、')
keywords2_list = keywords2.split('、')

# 加载预训练模型
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

# 对关键词进行编码以获取嵌入向量
embeddings1 = model.encode(keywords1_list, convert_to_tensor=True)
embeddings2 = model.encode(keywords2_list, convert_to_tensor=True)

# 计算关键词之间的余弦相似度
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

# 创建DataFrame并添加相似度矩阵
similarity_df = pd.DataFrame(cosine_scores.cpu().numpy())

# 设置相似度阈值
threshold = 0.7

# 创建标志列
flagged_series = similarity_df.apply(lambda x: 1 if (x > threshold).any() else 0)

# 将标志列转换为DataFrame
flagged_df = pd.DataFrame(flagged_series, columns=['相似詞'])

# 打印相似度矩阵
print("相似度矩陣：")
display(similarity_df)

# 打印标志列
print("標記：")
display(flagged_df)

num_ones = flagged_series.sum()

# 检查如果1的数量少于一半，输出"不够深入"
if num_ones < len(flagged_series) / 2:
    print("還有許多重點尚未探討到，繼續加油喔!")
elif num_ones < len(flagged_series) / 4:
    print("討論內容有點偏題了，有什麼需要幫忙的嗎？")

相似度矩陣：


,0,1,2,3,4
0,0.831008,0.704688,0.455559,0.447685,0.320219
1,0.663895,0.716362,0.260636,0.315838,0.164425
2,0.721420,0.434439,0.388585,0.321389,0.223621
3,0.553203,0.432848,0.300483,0.328760,0.123623
4,0.366995,0.383427,0.634148,0.372137,0.264406


標記：


,相似詞
0,1
1,1
2,0
3,0
4,0


還有許多重點尚未討論到，繼續加油喔!


In [6]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# from scipy.spatial.distance import cosine

# # 句子
# sentences = ['太陽能', '光']

# # 加载分词器和模型
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
# model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)


相似度： 0.3808121979236603
